In [16]:

import pandas as pd
from utils import unit_convert
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import os
import json

def load_trajectory(path):
    with open(path, 'r') as f:
        traj = json.load(f)

    stations = traj['trajectoryStation']

    res = {
        'mds': [],
        'tvds': [],
        'incls': []
    }

    for station in stations:
        res['tvds'].append(float(station['tvd']['value']))
        res['incls'].append(float(station['incl']['value']) * unit_convert('rad', 'degree'))
        res['mds'].append(float(station['md']['value']))

    return res

def get_data_from_download_format():
    local_res_path = fr"C:\NotOneDrive\Data\validate_tndsimulation\{well_name}.csv"

    local_tnd_o = pd.read_csv(local_res_path, skiprows=[1])
    local_tnd_o = local_tnd_o.rename(columns={'UtcTime':'time_key', 
                        'DataGenerator.HOOK_LOAD': 'hookload', 
                        'TND.HKLD_HIGH_IND': 'indicator', 
                        'TND.HKLD_PU_MODEL': 'pickup_hookload_model', 
                        'TND.HKLD_SO_MODEL':'slackoff_hookload_model',
                        'DepthMonitoring.RBD': 'bit_depth',
                        'DrillBoreHole.TD': 'total_depth'})
    local_tnd_o['time_key'] = pd.to_datetime(local_tnd_o['time_key'], utc=True)

    return local_tnd_o

def get_data_from_local_result_format():
    local_input_channel_path = r"C:\NotOneDrive\Data\algo_data_nas_copy\BDC 4-02\Run 5_c70ffa5d-c387-4c7b-97bf-420f1b85d09f\Channels.csv"

    local_input = pd.read_csv(local_input_channel_path, skiprows=[1]).rename(
        columns={
            'Time': 'time_key',
            'HOOK_LOAD': 'hookload',
            'DEPTH_BD': 'bit_depth',
            'DEPTH_TD': 'total_depth',
            'HOOK_HEIGHT': 'hook_height'
        }
    )[['time_key', 'hookload', 'bit_depth', 'total_depth', 'hook_height']]
    local_input['time_key'] = pd.to_datetime(local_input['time_key'], utc=True)

    local_res_path = fr"C:\NotOneDrive\Data\merged_input_output_channels\{well_name}.csv"

    local_tnd_o = pd.read_csv(local_res_path)
    local_tnd_o = local_tnd_o.rename(columns={
                        'high_indicator': 'indicator', 
                        'pickup_hkld': 'pickup_hookload_model', 
                        'slackoff_hkld':'slackoff_hookload_model',
                        })
    local_tnd_o['time_key'] = pd.to_datetime(local_tnd_o['time_key'], utc=True)

    local_tnd_o = pd.merge(local_input, local_tnd_o, on='time_key', how='outer')

    return local_tnd_o

well_name = 'BDC-4-02_bha_run5_20240802094250_TndSimulation'
# traj_path = r"C:\NotOneDrive\Data\algo_data_nas_copy\BDC 4-02\Run 5_c70ffa5d-c387-4c7b-97bf-420f1b85d09f\Trajectory_0.json"

local_tnd_o = get_data_from_local_result_format()

save_dir = r"C:\NotOneDrive\Data\validate_tndsimulation"
save_path = os.path.join(save_dir, well_name + '_TndSimulation' + '.html')
event_times = {}

if 'Dancer' in well_name:

    event_times = {
        '12/15/2021  5:39:00 PM' : 'stuck',
        '12/15/2021  9:34:00 PM' : 'potential pack off',
        '1/4/2022  6:12:00 AM': 'tight spots',
        '1/9/2022  1:55:00 PM' : "LOT",
        '1/10/2022 2:55:00 PM' : 'stuck',
        '1/10/2022 11:04:00 PM' : 'high torque',
        '1/11/2022 1:14:00 PM' : 'over pull',
        '1/13/2022 8:14:00 AM' : 'pack off',
        '1/14/2022 4:05:00 AM' : 'stuck inside shoe'
    }
for k , v in list(event_times.items()):
    event_times[pd.to_datetime(k, utc=True) - pd.Timedelta(hours=8)] = v
    del event_times[k]

local_tnd_o.dropna(how='all', inplace=True, subset=['hookload', 'bit_depth', 'total_depth', 'indicator', 'pickup_hookload_model', 'slackoff_hookload_model', 'hook_height'])
# local_tnd_o.to_csv('test.csv', index=False)
local_tnd_o = local_tnd_o[local_tnd_o['hook_height'] < 50]
local_tnd_o = local_tnd_o[local_tnd_o['hook_height'] > -50]

# traj = load_trajectory(traj_path)


In [ ]:
sample_rate = 5


local_tnd = local_tnd_o.iloc[::sample_rate]


marker_size = 4
line_width = 0.5

fig = make_subplots(rows = 3, cols = 1,
                        specs=[
                            [{"secondary_y": True}],
                            [{"secondary_y": True}],
                            [{"secondary_y": True}],
                            ],
                            shared_xaxes=True)

fig.add_trace(
    go.Scattergl(
        mode = 'markers',
        marker = dict(
            size = marker_size
        ),
        line={'width': line_width},
        x=local_tnd['time_key'], 
        y=local_tnd['hookload'], 
        name="measured hkld"),
        row=1, col=1,
)

pu_hkld = local_tnd_o[['pickup_hookload_model', 'time_key']].dropna()
print(pu_hkld)

fig.add_trace(
    go.Scattergl(
        mode = 'markers',
        marker = dict(
            size = marker_size
        ),
        line={'width': line_width},
        x=local_tnd['time_key'], 
        y=local_tnd['pickup_hookload_model'], 
        name="modeled pickup hkld"),
        row=1, col=1,
)

fig.add_trace(
    go.Scattergl(
        mode = 'markers',
        marker = dict(
            size = marker_size
        ),
        line={'width': line_width},
        x=local_tnd['time_key'], 
        y=local_tnd['slackoff_hookload_model'], 
        name="modeled slackoff hkld"),
        row=1, col=1,
)

# fig['layout']['yaxis']['title'] = 'Hookload (klbf)'

fig.add_trace(
    go.Scattergl(
        mode = 'markers',
        marker = dict(
            size = marker_size
        ),
        line={'width': line_width},
        x=local_tnd['time_key'], 
        y=local_tnd['indicator'], 
        name="modeled indicator",
        ),
        row=2, col=1,
)

fig.add_trace(
    go.Scattergl(
        mode = 'markers',
        marker = dict(
            size = marker_size
        ),
        line={'width': line_width},
        x=local_tnd['time_key'], 
        y=local_tnd['bit_depth'], 
        name="bit depth",
        ),
        row=3, col=1,
)

fig.add_trace(
    go.Scattergl(
        mode = 'markers',
        marker = dict(
            size = marker_size
        ),
        line={'width': line_width},
        x=local_tnd['time_key'], 
        y=local_tnd['total_depth'], 
        name="total depth",
        ),
        row=3, col=1,
)

fig.add_trace(
    go.Scattergl(
        mode = 'markers',
        marker = dict(
            size = marker_size
        ),
        line={'width': line_width},
        x=local_tnd['time_key'], 
        y=local_tnd['hook_height'], 
        name="hook height",
        ),
        row=3, col=1,
        secondary_y = True
)
    
fig['layout']['yaxis6']['title'] = 'Hook height'

# fig.add_trace(
#     go.Scattergl(
#         mode = 'lines',
#         line={'width': 2},
#         x=traj['incls'], 
#         y=traj['mds'], 
#         name="trajectory",
#         xaxis='x4'),
#         row=4, col=1,
# )

fig.add_hline(y = 0.5, row=2, col=1, line_color="green")
fig.add_hline(y = 1, row=2, col=1,line_color="red")

# fig['layout']['yaxis2']['title'] = 'Indicator (unitless)'

for t, v in event_times.items():
    if t < local_tnd['time_key'].iloc[-1] and t > local_tnd['time_key'].iloc[0]:
        print(t)
        fig.add_vrect(t,t,annotation_text=v, annotation_position="top left" )

fig.update_xaxes(
    tickfont=dict(size=20),
    showgrid=False, zeroline=False )

fig.update_yaxes(
    showgrid=False, 
    zeroline=False )
fig.update_layout(
    title = dict(
        text = fr'Tnd Simulation Validation - {well_name}',
        font = dict(size = 30)
    ),
    yaxis=dict(title=dict(
        text = "Hookload (kkgf)",
        font = dict(size = 15)
        ), 
        ),
    yaxis3=dict(title=
                dict(
                    text="Indicator (unitless)",
                    font=dict(size = 15)        
                )
        ),
    yaxis5=dict(title=
                dict(
                    text="Depth (m)",
                    font=dict(size = 15)        
                )
        ),
    xaxis3=dict(title=dict(
        text = "Time",
        font = dict(size = 20)
        )),
    legend = dict(font = dict(size = 20)),
        )

fig.write_html(save_path)
fig.show()